# Cleaning and Combining Data
In this notebook we will go through the downloaded and scrapped data to prepare it for exploritory data analysis.

## Load Dependencies

In [1]:
import unidecode
import csv
import pandas as pd
import re
import numpy as np

## Removing Diacritics From Excel
Since alot of Wine names have foreign words in it, which uses diacritics. If I were to use SQL for data exploration, I would need to clean up the diacritics.

In [2]:
# Sanity Check
wine_scores = pd.read_csv('data\intermediate\kaggle_wine_scores.csv')
wine_scores.head()

,id,country,designation,points,price,province,region_1,region_2,title,variety,winery
0,0,Italy,Vulkà Bianco,87,Null,Sicily & Sardinia,Etna,Null,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,Avidagos,87,15,Douro,Null,Null,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,Null,87,14,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,Reserve Late Harvest,87,13,Michigan,Lake Michigan Shore,Null,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,Vintner's Reserve Wild Child Block,87,65,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
wine_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           129971 non-null  int64 
 1   country      129971 non-null  object
 2   designation  129971 non-null  object
 3   points       129971 non-null  int64 
 4   price        129971 non-null  object
 5   province     129971 non-null  object
 6   region_1     129971 non-null  object
 7   region_2     129971 non-null  object
 8   title        129971 non-null  object
 9   variety      129971 non-null  object
 10  winery       129971 non-null  object
dtypes: int64(2), object(9)
memory usage: 10.9+ MB


In [4]:
# Define file paths
input_csv = 'data\intermediate\kaggle_wine_scores.csv' 
output_csv = 'data\intermediate\wine_scores_with_diacritics_removed.csv'

# Function to clean the strings by replacing diacritics with ASCII equivalents
def clean_string(s):
    return unidecode.unidecode(s)

# Open the input CSV file and create a new output CSV file
with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
    # Create a CSV reader and writer
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Read and write the header
    header = next(reader)
    writer.writerow(header)

    # Process each row in the CSV
    for row in reader:
        # Clean each cell in the row
        cleaned_row = [clean_string(cell) for cell in row]
        # Write the cleaned row to the output CSV
        writer.writerow(cleaned_row)

print('Diacritics converted to plain English. Converted file saved as:', output_csv)

Diacritics converted to plain English. Converted file saved as: data\intermediate\wine_scores_with_diacritics_removed.csv


In [5]:
# Sanity Check
df = pd.read_csv('data\intermediate\wine_scores_with_diacritics_removed.csv')
df.head()

,id,country,designation,points,price,province,region_1,region_2,title,variety,winery
0,0,Italy,Vulka Bianco,87,Null,Sicily & Sardinia,Etna,Null,Nicosia 2013 Vulka Bianco (Etna),White Blend,Nicosia
1,1,Portugal,Avidagos,87,15,Douro,Null,Null,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,Null,87,14,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,Reserve Late Harvest,87,13,Michigan,Lake Michigan Shore,Null,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,Vintner's Reserve Wild Child Block,87,65,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           129971 non-null  int64 
 1   country      129971 non-null  object
 2   designation  129971 non-null  object
 3   points       129971 non-null  int64 
 4   price        129971 non-null  object
 5   province     129971 non-null  object
 6   region_1     129971 non-null  object
 7   region_2     129971 non-null  object
 8   title        129971 non-null  object
 9   variety      129971 non-null  object
 10  winery       129971 non-null  object
dtypes: int64(2), object(9)
memory usage: 10.9+ MB


## Extract Vintage information from title column

In [7]:
# Function to extract vintage from the title
def extract_vintage(title):
    # Regular expression to find four consecutive digits
    match = re.search(r'(\b\d{4}\b)', title)
    # If a match is found, return the match, otherwise return None
    return int(match.group(1)) if match else pd.NA

# Apply the function to the 'title' column and create a new 'vintage' column
df['vintage'] = df['title'].apply(extract_vintage).astype('Int64')

# Save the modified DataFrame to a new CSV file
df.to_csv('data\intermediate\wine_score_with_vintage.csv', index=False)
print('New CSV with vintage column saved as: wine_score_with_vintage.csv')

New CSV with vintage column saved as: wine_score_with_vintage.csv


In [8]:
#Sanity Check
df.head()

,id,country,designation,points,price,province,region_1,region_2,title,variety,winery,vintage
0,0,Italy,Vulka Bianco,87,Null,Sicily & Sardinia,Etna,Null,Nicosia 2013 Vulka Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,Avidagos,87,15,Douro,Null,Null,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,Null,87,14,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,3,US,Reserve Late Harvest,87,13,Michigan,Lake Michigan Shore,Null,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,4,US,Vintner's Reserve Wild Child Block,87,65,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           129971 non-null  int64 
 1   country      129971 non-null  object
 2   designation  129971 non-null  object
 3   points       129971 non-null  int64 
 4   price        129971 non-null  object
 5   province     129971 non-null  object
 6   region_1     129971 non-null  object
 7   region_2     129971 non-null  object
 8   title        129971 non-null  object
 9   variety      129971 non-null  object
 10  winery       129971 non-null  object
 11  vintage      125362 non-null  Int64 
dtypes: Int64(1), int64(2), object(9)
memory usage: 12.0+ MB


## Change "Null" string value to PANDAS compatible null value

In [10]:
# Replace 'Null' strings with numpy.nan across the entire DataFrame
df.replace('Null', np.nan, inplace=True)

In [11]:
# Sanity Check
df.head()

,id,country,designation,points,price,province,region_1,region_2,title,variety,winery,vintage
0,0,Italy,Vulka Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulka Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,Avidagos,87,15,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,NaN,87,14,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,3,US,Reserve Late Harvest,87,13,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,4,US,Vintner's Reserve Wild Child Block,87,65,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           129971 non-null  int64 
 1   country      129908 non-null  object
 2   designation  92506 non-null   object
 3   points       129971 non-null  int64 
 4   price        120975 non-null  object
 5   province     129908 non-null  object
 6   region_1     108724 non-null  object
 7   region_2     50511 non-null   object
 8   title        129971 non-null  object
 9   variety      129970 non-null  object
 10  winery       129971 non-null  object
 11  vintage      125362 non-null  Int64 
dtypes: Int64(1), int64(2), object(9)
memory usage: 12.0+ MB


In [13]:
# Convert price to Int64
df['price'] = df['price'].astype('Int64')
# Check Dtype
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           129971 non-null  int64 
 1   country      129908 non-null  object
 2   designation  92506 non-null   object
 3   points       129971 non-null  int64 
 4   price        120975 non-null  Int64 
 5   province     129908 non-null  object
 6   region_1     108724 non-null  object
 7   region_2     50511 non-null   object
 8   title        129971 non-null  object
 9   variety      129970 non-null  object
 10  winery       129971 non-null  object
 11  vintage      125362 non-null  Int64 
dtypes: Int64(2), int64(2), object(8)
memory usage: 12.1+ MB


In [14]:
#Recheck Dataframe
df.head()

,id,country,designation,points,price,province,region_1,region_2,title,variety,winery,vintage
0,0,Italy,Vulka Bianco,87,<NA>,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulka Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,Avidagos,87,15,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,NaN,87,14,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,3,US,Reserve Late Harvest,87,13,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,4,US,Vintner's Reserve Wild Child Block,87,65,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [15]:
# Export to CSV
df.to_csv('data\clean\cleaned_wine_score.csv', index=False)
print('New CSV with cleaned data saved as: cleaned_wine_score.csv')

New CSV with cleaned data saved as: cleaned_wine_score.csv


In [16]:
# Load Dataframe
df = pd.read_csv('data\clean\cleaned_wine_score.csv')

# Check null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           129971 non-null  int64  
 1   country      129908 non-null  object 
 2   designation  92506 non-null   object 
 3   points       129971 non-null  int64  
 4   price        120975 non-null  float64
 5   province     129908 non-null  object 
 6   region_1     108724 non-null  object 
 7   region_2     50511 non-null   object 
 8   title        129971 non-null  object 
 9   variety      129970 non-null  object 
 10  winery       129971 non-null  object 
 11  vintage      125362 non-null  float64
dtypes: float64(2), int64(2), object(8)
memory usage: 11.9+ MB


## Cleaning Scrapped Data

In [17]:
# Load Scrapped Data
scraped = pd.read_csv('webscrapper\output\wine_info.csv')
scraped.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Wine Type
0,Benovia 2021 Chardonnay (Russian River Valley),Sonoma,California,NaN,US,93,$48,Benovia,Chardonnay,White
1,Benovia 2021 Pinot Noir (Russian River Valley),Sonoma,California,NaN,US,92,$55,Benovia,Pinot Noir,Red
2,Benovia 2020 Cohn Vineyard Pinot Noir (Sonoma ...,Sonoma,California,NaN,US,95,$100,Benovia,Pinot Noir,Red
3,Ross Knoll Vineyard 2021 Pinot Noir (Russian R...,Sonoma,California,NaN,US,92,$60,Ross Knoll Vineyard,Pinot Noir,Red
4,Belle Glos 2022 Clark & Telephone Pinot Noir (...,Central Coast,California,NaN,US,92,$55,Belle Glos,Pinot Noir,Red


In [18]:
scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42479 entries, 0 to 42478
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Wine Name  42479 non-null  object
 1   Region 1   34635 non-null  object
 2   Region 2   23504 non-null  object
 3   Region 3   16229 non-null  object
 4   Country    42479 non-null  object
 5   Score      42479 non-null  int64 
 6   Price      38849 non-null  object
 7   Winery     42479 non-null  object
 8   Variety    40626 non-null  object
 9   Wine Type  42479 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.2+ MB


In [19]:
# Function to extract vintage from the title
def extract_vintage(title):
    # Regular expression to find four consecutive digits
    match = re.search(r'(\b\d{4}\b)', title)
    # If a match is found, return the match, otherwise return None
    return int(match.group(1)) if match else pd.NA

# Apply the function to the 'title' column and create a new 'vintage' column
scraped['vintage'] = scraped['Wine Name'].apply(extract_vintage).astype('Int64')

# Save the modified DataFrame to a new CSV file
scraped.to_csv('data\intermediate\scraped_info_with_vintage.csv', index=False)
print('New CSV with vintage column saved as: scraped_info_with_vintage.csv')

New CSV with vintage column saved as: scraped_info_with_vintage.csv


In [20]:
scraped.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Wine Type,vintage
0,Benovia 2021 Chardonnay (Russian River Valley),Sonoma,California,NaN,US,93,$48,Benovia,Chardonnay,White,2021
1,Benovia 2021 Pinot Noir (Russian River Valley),Sonoma,California,NaN,US,92,$55,Benovia,Pinot Noir,Red,2021
2,Benovia 2020 Cohn Vineyard Pinot Noir (Sonoma ...,Sonoma,California,NaN,US,95,$100,Benovia,Pinot Noir,Red,2020
3,Ross Knoll Vineyard 2021 Pinot Noir (Russian R...,Sonoma,California,NaN,US,92,$60,Ross Knoll Vineyard,Pinot Noir,Red,2021
4,Belle Glos 2022 Clark & Telephone Pinot Noir (...,Central Coast,California,NaN,US,92,$55,Belle Glos,Pinot Noir,Red,2022


In [21]:
# Reorganizing downloaded data

# First, drop the 'id' and 'designation' columns
df.drop(columns=['id', 'designation'], inplace=True)

# Then, rename the columns
df.rename(columns={
    'title': 'Wine Name',
    'region_1': 'Region 1',
    'province': 'Region 2',
    'region_2': 'Region 3',
    'country': 'Country',
    'points': 'Score',
    'price': 'Price',
    'winery': 'Winery',
    'variety': 'Variety',
    'vintage': 'Vintage'
}, inplace=True)

# Finally, reorder the columns
df = df[['Wine Name', 'Region 1', 'Region 2', 'Region 3', 'Country', 'Score', 'Price', 'Winery', 'Variety', 'Vintage']]

# Check results
df.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,Nicosia 2013 Vulka Bianco (Etna),Etna,Sicily & Sardinia,NaN,Italy,87,NaN,Nicosia,White Blend,2013.0
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),NaN,Douro,NaN,Portugal,87,15.0,Quinta dos Avidagos,Portuguese Red,2011.0
2,Rainstorm 2013 Pinot Gris (Willamette Valley),Willamette Valley,Oregon,Willamette Valley,US,87,14.0,Rainstorm,Pinot Gris,2013.0
3,St. Julian 2013 Reserve Late Harvest Riesling ...,Lake Michigan Shore,Michigan,NaN,US,87,13.0,St. Julian,Riesling,2013.0
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Willamette Valley,Oregon,Willamette Valley,US,87,65.0,Sweet Cheeks,Pinot Noir,2012.0


In [22]:
# Save the modified DataFrame to a new CSV file
df.to_csv('data\intermediate\standardized_downloaded_wine_data.csv', index=False)
print('New CSV with vintage column saved as: standardized_downloaded_wine_data.csv')

New CSV with vintage column saved as: standardized_downloaded_wine_data.csv


In [23]:
scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42479 entries, 0 to 42478
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Wine Name  42479 non-null  object
 1   Region 1   34635 non-null  object
 2   Region 2   23504 non-null  object
 3   Region 3   16229 non-null  object
 4   Country    42479 non-null  object
 5   Score      42479 non-null  int64 
 6   Price      38849 non-null  object
 7   Winery     42479 non-null  object
 8   Variety    40626 non-null  object
 9   Wine Type  42479 non-null  object
 10  vintage    37354 non-null  Int64 
dtypes: Int64(1), int64(1), object(9)
memory usage: 3.6+ MB


In [24]:
# Now we standardize the scraped data
# Drop the 'Wine Type' column
scraped.drop(columns=['Wine Type'], inplace=True)

# Rename 'vintage' to 'Vintage'
scraped.rename(columns={'vintage': 'Vintage'}, inplace=True)

# Check Results
scraped.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,Benovia 2021 Chardonnay (Russian River Valley),Sonoma,California,NaN,US,93,$48,Benovia,Chardonnay,2021
1,Benovia 2021 Pinot Noir (Russian River Valley),Sonoma,California,NaN,US,92,$55,Benovia,Pinot Noir,2021
2,Benovia 2020 Cohn Vineyard Pinot Noir (Sonoma ...,Sonoma,California,NaN,US,95,$100,Benovia,Pinot Noir,2020
3,Ross Knoll Vineyard 2021 Pinot Noir (Russian R...,Sonoma,California,NaN,US,92,$60,Ross Knoll Vineyard,Pinot Noir,2021
4,Belle Glos 2022 Clark & Telephone Pinot Noir (...,Central Coast,California,NaN,US,92,$55,Belle Glos,Pinot Noir,2022


In [25]:
# Save the modified DataFrame to a new CSV file
df.to_csv('data\intermediate\standardized_scraped_wine_data.csv', index=False)
print('New CSV with vintage column saved as: standardized_scraped_wine_data.csv')

New CSV with vintage column saved as: standardized_scraped_wine_data.csv


## Combining and Cleaning Combined Data Frame

In [26]:
# Combine 'df' and 'scraped' DataFrames
combined_df = pd.concat([df, scraped])

# Sort the combined DataFrame by 'Vintage' in increasing order
combined_df.sort_values(by='Vintage', inplace=True)

# Reset the index of the sorted DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Check Results
combined_df.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,1000 Stories 2014 Bourbon Barrel Aged Batch No...,California,California,California Other,US,90,19.0,1000 Stories,Zinfandel,1000.0
1,1000 Stories 2013 Bourbon Barrel Aged Zinfande...,Mendocino,California,NaN,US,91,19.0,1000 Stories,Zinfandel,1000.0
2,1070 Green 2011 Sauvignon Blanc (Rutherford),Rutherford,California,Napa,US,88,25.0,1070 Green,Sauvignon Blanc,1070.0
3,Ikal 1150 2007 Chardonnay (Tupungato),Tupungato,Mendoza Province,NaN,Argentina,83,20.0,Ikal 1150,Chardonnay,1150.0
4,Ikal 1150 2007 Malbec (Tupungato),Tupungato,Mendoza Province,NaN,Argentina,86,20.0,Ikal 1150,Malbec,1150.0


In [27]:
# Changing all -1 values in the 'Vintage' column to null (NaN) values
combined_df['Vintage'].replace(-1, pd.NA, inplace=True)

# Check Results
combined_df.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,1000 Stories 2014 Bourbon Barrel Aged Batch No...,California,California,California Other,US,90,19.0,1000 Stories,Zinfandel,1000.0
1,1000 Stories 2013 Bourbon Barrel Aged Zinfande...,Mendocino,California,NaN,US,91,19.0,1000 Stories,Zinfandel,1000.0
2,1070 Green 2011 Sauvignon Blanc (Rutherford),Rutherford,California,Napa,US,88,25.0,1070 Green,Sauvignon Blanc,1070.0
3,Ikal 1150 2007 Chardonnay (Tupungato),Tupungato,Mendoza Province,NaN,Argentina,83,20.0,Ikal 1150,Chardonnay,1150.0
4,Ikal 1150 2007 Malbec (Tupungato),Tupungato,Mendoza Province,NaN,Argentina,86,20.0,Ikal 1150,Malbec,1150.0


In [28]:
# Save the modified DataFrame to a new CSV file
combined_df.to_csv('data\intermediate\combined_wine_data.csv', index=False)
print('New CSV with vintage column saved as: combined_wine_data.csv')

New CSV with vintage column saved as: combined_wine_data.csv


In [29]:
# Now that the scrapped data is combined with the downloaded data, there may still be diacritics that need to be cleaned.
# Define file paths
input_csv = 'data\intermediate\combined_wine_data.csv' 
output_csv = 'data\intermediate\combined_wine_data_with_diacritics_removed.csv'

# Function to clean the strings by replacing diacritics with ASCII equivalents
def clean_string(s):
    return unidecode.unidecode(s)

# Open the input CSV file and create a new output CSV file
with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
    # Create a CSV reader and writer
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Read and write the header
    header = next(reader)
    writer.writerow(header)

    # Process each row in the CSV
    for row in reader:
        # Clean each cell in the row
        cleaned_row = [clean_string(cell) for cell in row]
        # Write the cleaned row to the output CSV
        writer.writerow(cleaned_row)

print('Diacritics converted to plain English. Converted file saved as:', output_csv)

Diacritics converted to plain English. Converted file saved as: data\intermediate\combined_wine_data_with_diacritics_removed.csv


In [30]:
combined_df = pd.read_csv('data\intermediate\combined_wine_data_with_diacritics_removed.csv')
combined_df.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,1000 Stories 2014 Bourbon Barrel Aged Batch No...,California,California,California Other,US,90,19.0,1000 Stories,Zinfandel,1000.0
1,1000 Stories 2013 Bourbon Barrel Aged Zinfande...,Mendocino,California,NaN,US,91,19.0,1000 Stories,Zinfandel,1000.0
2,1070 Green 2011 Sauvignon Blanc (Rutherford),Rutherford,California,Napa,US,88,25.0,1070 Green,Sauvignon Blanc,1070.0
3,Ikal 1150 2007 Chardonnay (Tupungato),Tupungato,Mendoza Province,NaN,Argentina,83,20.0,Ikal 1150,Chardonnay,1150.0
4,Ikal 1150 2007 Malbec (Tupungato),Tupungato,Mendoza Province,NaN,Argentina,86,20.0,Ikal 1150,Malbec,1150.0


In [31]:
# Since we can only use data that have a vintage and we only have weather data as early as 1961, we need to run the following
# script to make sure there isn't any incorrect vintages and drop all vintages that is showing up earlier than 1961.

def re_extract_vintage_adjusted_v2(row):
    # Regular expression to find four-digit numbers
    matches = re.findall(r'\b\d{4}\b', row['Wine Name'])
    # Filter out numbers less than 1961
    valid_matches = [int(match) for match in matches if int(match) >= 1961]
    
    if valid_matches:
        # Return the first valid match if found
        return valid_matches[0]
    # If no valid match is found, return pd.NA to indicate dropping the row later
    return pd.NA

# Apply the function to all rows in the DataFrame
combined_df['Vintage'] = combined_df.apply(re_extract_vintage_adjusted_v2, axis=1)

# Drop rows where 'Vintage' is null or less than 1961
combined_df = combined_df.dropna(subset=['Vintage'])
combined_df = combined_df[combined_df['Vintage'] >= 1961]

# Sort the combined DataFrame by 'Vintage' in increasing order
combined_df.sort_values(by='Vintage', inplace=True)

# Reset the index of the DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Check results
combined_df.head()

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,Calem 1961 Colheita Tawny Port (Port),NaN,Port,NaN,Portugal,95,$320,Calem,Port Blend,1961
1,Calem 1961 Colheita Tawny (Port),NaN,Port,NaN,Portugal,95,320.0,Calem,Port,1961
2,Warre's 1961 Reserve Tawny Port (Port),NaN,Port,NaN,Portugal,89,$111,Warre's,Port Blend,1961
3,Wiese & Krohn 1961 Colheita Port (Port),NaN,Port,NaN,Portugal,92,$200,Wiese & Krohn,Port Blend,1961
4,Cossart Gordon 1962 Bual (Madeira),NaN,Madeira,NaN,Portugal,96,$355,Cossart Gordon,Madeira,1962


In [32]:
# Price data also need to be standardized
def clean_price(price):
    if pd.isna(price):
        return price
    # Remove $ sign if present and convert to float
    return float(price.replace('$', ''))

# Apply the function to the 'Price' column
combined_df['Price'] = combined_df['Price'].apply(clean_price)

# Format the 'Price' column to show value up to the second decimal place
combined_df['Price'] = combined_df['Price'].map('{:.2f}'.format)

# Sort the combined DataFrame by 'Vintage' in increasing order
combined_df.sort_values(by='Vintage', inplace=True)

# Reset the index of the DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Check results
combined_df.head(10)

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,Calem 1961 Colheita Tawny Port (Port),NaN,Port,NaN,Portugal,95,320.00,Calem,Port Blend,1961
1,Calem 1961 Colheita Tawny (Port),NaN,Port,NaN,Portugal,95,320.00,Calem,Port,1961
2,Warre's 1961 Reserve Tawny Port (Port),NaN,Port,NaN,Portugal,89,111.00,Warre's,Port Blend,1961
3,Wiese & Krohn 1961 Colheita Port (Port),NaN,Port,NaN,Portugal,92,200.00,Wiese & Krohn,Port Blend,1961
4,Cossart Gordon 1962 Bual (Madeira),NaN,Madeira,NaN,Portugal,96,355.00,Cossart Gordon,Madeira,1962
5,Van Zellers 1962 Palmer Colheita White Port (P...,NaN,Port,NaN,Portugal,90,300.00,Van Zellers,Port Blend,1962
6,Quinta da Pacheca 1962 Pacheca Colheita Old Ta...,NaN,Port,NaN,Portugal,95,500.00,Quinta da Pacheca,Port Blend,1962
7,Taylor Fladgate 1963 Vintage Port (Port),NaN,Port,NaN,Portugal,92,430.00,Taylor Fladgate,Port Blend,1963
8,Dow's 1963 Vintage Port (Port),NaN,Port,NaN,Portugal,95,467.00,Dow's,Port Blend,1963
9,Barros 1963 Colheita Port (Port),NaN,Port,NaN,Portugal,91,215.00,Barros,Port Blend,1963


In [33]:
# Dropping duplicate rows based on the 'Wine Name' column
combined_df = combined_df.drop_duplicates(subset=['Wine Name'])

# Sort the combined DataFrame by 'Vintage' in increasing order
combined_df.sort_values(by='Vintage', inplace=True)

# Reset the index of the DataFrame
combined_df.reset_index(drop=True, inplace=True)

# Check results
combined_df.head(10)

,Wine Name,Region 1,Region 2,Region 3,Country,Score,Price,Winery,Variety,Vintage
0,Calem 1961 Colheita Tawny Port (Port),NaN,Port,NaN,Portugal,95,320.00,Calem,Port Blend,1961
1,Calem 1961 Colheita Tawny (Port),NaN,Port,NaN,Portugal,95,320.00,Calem,Port,1961
2,Warre's 1961 Reserve Tawny Port (Port),NaN,Port,NaN,Portugal,89,111.00,Warre's,Port Blend,1961
3,Wiese & Krohn 1961 Colheita Port (Port),NaN,Port,NaN,Portugal,92,200.00,Wiese & Krohn,Port Blend,1961
4,Cossart Gordon 1962 Bual (Madeira),NaN,Madeira,NaN,Portugal,96,355.00,Cossart Gordon,Madeira,1962
5,Van Zellers 1962 Palmer Colheita White Port (P...,NaN,Port,NaN,Portugal,90,300.00,Van Zellers,Port Blend,1962
6,Quinta da Pacheca 1962 Pacheca Colheita Old Ta...,NaN,Port,NaN,Portugal,95,500.00,Quinta da Pacheca,Port Blend,1962
7,Burmester 1963 Colheita (Port),NaN,Port,NaN,Portugal,87,790.00,Burmester,Port,1963
8,Cockburn's 1963 Vintage Port (Port),NaN,Port,NaN,Portugal,90,nan,Cockburn's,Port Blend,1963
9,Dow's 1963 Vintage (Port),NaN,Port,NaN,Portugal,95,467.00,Dow's,Port,1963


In [34]:
# Check remaining rows of data
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143278 entries, 0 to 143277
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Wine Name  143278 non-null  object
 1   Region 1   119127 non-null  object
 2   Region 2   128970 non-null  object
 3   Region 3   56225 non-null   object
 4   Country    143221 non-null  object
 5   Score      143278 non-null  int64 
 6   Price      143278 non-null  object
 7   Winery     143278 non-null  object
 8   Variety    141662 non-null  object
 9   Vintage    143278 non-null  object
dtypes: int64(1), object(9)
memory usage: 10.9+ MB


In [35]:
# Save the modified DataFrame to a new CSV file
combined_df.to_csv('data\clean\cleaned_combined_wine_data.csv', index=False)
print('New CSV with vintage column saved as: cleaned_combined_wine_data.csv')

New CSV with vintage column saved as: cleaned_combined_wine_data.csv


## Clean Weather Data
The weather data I have is in a raw .txt format so I would need to write a script that will convert it into workable csv files.

In [43]:
# Proof of concept parsing and txt to csv conversion script
def parse_value(row, start, data_type):
    value_str = row[start:start+5].replace('/', '').strip()  # Remove slashes
    if '-' in value_str:
        # If there's a dash, remove it and treat the number as negative
        value_str = value_str.replace('-', '')
        value = -float(value_str) if value_str else None
    else:
        value = float(value_str) if value_str else None
    
    return round(value * 0.1, 1) if value is not None and data_type != "1" else value

def parse_row(row, station_info):
    station_id = row[2:7]
    data_type = row[7]
    if data_type == "1":
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = row[8:12]
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)

# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\sample1.txt', r'data\raw\sample1.csv')

In [44]:
# Check results
sample_weather_data = pd.read_csv(r'data\raw\sample1.csv')
sample_weather_data

,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1001,NORWAY,JAN MAYEN,4,2011,-3.3,-0.6,-5.9,-0.5,1.0,3.5,5.3,6.1,5.9,2.1,1.3,-3.3
1,1001,NORWAY,JAN MAYEN,4,2012,0.5,-3.7,-2.3,-4.5,-0.6,2.4,4.8,5.9,3.3,1.0,-1.1,-2.1
2,1001,NORWAY,JAN MAYEN,5,2011,112.2,99.3,65.7,83.9,30.3,27.5,12.6,34.7,108.5,79.1,153.3,56.7
3,1001,NORWAY,JAN MAYEN,5,2012,61.7,78.0,59.5,12.0,24.8,6.5,18.9,32.2,77.0,34.9,92.1,42.8
4,2013,SWEDEN,RITSEM A,2,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,SWEDEN,RITSEM A,2,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,SWEDEN,RITSEM A,2,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,SWEDEN,RITSEM A,2,2014,949.4,934.5,938.8,945.8,951.6,950.9,953.1,NaN,949.5,NaN,950.8,931.2
8,2013,SWEDEN,RITSEM A,2,2015,932.3,933.6,943.4,941.0,942.2,943.9,944.4,952.7,950.7,949.8,931.9,934.2
9,2013,SWEDEN,RITSEM A,2,2016,941.7,931.3,943.5,944.2,951.6,949.7,945.5,946.7,947.2,961.9,943.8,937.7


The parsing script seems to work for the sample data. We can now use this to run through all weather data sets. Just a quick legend on what the data type number represents:
- **1:** Station Information - This is not represented in the CSV as it's own row since the information is intergrated into every row. This is the weather station ID, the country and city where the weather station is at.
- **2:** Mean Station Pressure (in hPa)
- **3:** Mean Sea Level Pressure (in hPa)
- **4:** Mean Daily Air Temperature (in deg Celsius)
- **5:** Total Monthly Precipitation (in millimeters)
- **6:** Mean Daily Maximum Air Temperature (in deg Celsius)
- **7:** Mean Daily Minimum Air Temperature (in deg Celsius)
- **8:** Mean Daily Relative Humidity (in percent)

In [45]:
# 1961-1970 Weather Data Parsing
def parse_value(row, start, data_type):
    value_str = row[start:start+5].replace('/', '').strip()  # Remove slashes
    if '-' in value_str:
        # If there's a dash, remove it and treat the number as negative
        value_str = value_str.replace('-', '')
        value = -float(value_str) if value_str else None
    else:
        value = float(value_str) if value_str else None
    
    return round(value * 0.1, 1) if value is not None and data_type != "1" else value

def parse_row(row, station_info):
    station_id = row[2:7]
    data_type = row[7]
    if data_type == "1":
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = row[8:12]
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)
        
# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\WWR_v01_global_array_1961_1970.txt', r'data\intermediate\1961_1970_weather_data.csv')

In [46]:
# Check results
weather_data_1961_1970 = pd.read_csv(r'data\intermediate\1961_1970_weather_data.csv')
weather_data_1961_1970.head()

,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,43333,ANDAMAN ISLANDS,PORT BLAIR,3,1961,1013.0,1010.0,1012.0,1009.0,1007.0,1007.0,1008.0,1007.0,1009.0,1011.0,1011.0,1011.0
1,43333,ANDAMAN ISLANDS,PORT BLAIR,3,1962,1011.0,1013.0,1011.0,1010.0,1007.0,1007.0,1007.0,1009.0,1008.0,1010.0,1012.0,1012.0
2,43333,ANDAMAN ISLANDS,PORT BLAIR,3,1963,1012.0,1013.0,1011.0,1010.1,1007.0,1007.9,1007.4,1007.9,1009.1,1011.1,1011.6,1012.5
3,43333,ANDAMAN ISLANDS,PORT BLAIR,3,1964,1013.0,1011.6,1011.5,1008.9,1006.9,1006.9,1007.7,1007.0,1007.4,1009.1,1009.7,1012.3
4,43333,ANDAMAN ISLANDS,PORT BLAIR,3,1965,1013.3,1010.7,1011.7,1009.9,1006.7,1007.7,1008.2,1008.3,1009.1,1010.5,1012.0,1010.5


In [47]:
# 1971-1980 Weather Data Parsing
def parse_value(row, start, data_type):
    value_str = row[start:start+5].replace('/', '').strip()  # Remove slashes
    if '-' in value_str:
        # If there's a dash, remove it and treat the number as negative
        value_str = value_str.replace('-', '')
        value = -float(value_str) if value_str else None
    else:
        value = float(value_str) if value_str else None
    
    return round(value * 0.1, 1) if value is not None and data_type != "1" else value

def parse_row(row, station_info):
    station_id = row[2:7]
    data_type = row[7]
    if data_type == "1":
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = row[8:12]
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)
        
# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\WWR_v01_global_array_1971-1980.txt', r'data\intermediate\1971_1980_weather_data.csv')

In [48]:
# Check results
weather_data_1971_1980 = pd.read_csv(r'data\intermediate\1971_1980_weather_data.csv')
weather_data_1971_1980.head()

,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,60620,ALGERIA,ADRAR,3,1971,1019.0,1017.0,1010.0,1008.0,1007.0,1007.8,NaN,1007.0,1010.0,1017.0,NaN,NaN
1,60620,ALGERIA,ADRAR,3,1972,NaN,NaN,NaN,1008.0,1008.0,1006.0,NaN,1006.0,1011.0,1014.0,1016.0,1020.0
2,60620,ALGERIA,ADRAR,3,1973,1022.0,1016.0,1014.0,1010.0,1009.0,1006.0,NaN,1008.0,NaN,1013.0,1018.0,1017.0
3,60620,ALGERIA,ADRAR,3,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60620,ALGERIA,ADRAR,3,1975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# 1981-1990 Weather Data Parsing
def parse_value(row, start, data_type):
    value_str = row[start:start+5].replace('/', '').strip()  # Remove slashes
    if '-' in value_str:
        # If there's a dash, remove it and treat the number as negative
        value_str = value_str.replace('-', '')
        value = -float(value_str) if value_str else None
    else:
        value = float(value_str) if value_str else None
    
    return round(value * 0.1, 1) if value is not None and data_type != "1" else value

def parse_row(row, station_info):
    station_id = row[2:7]
    data_type = row[7]
    if data_type == "1":
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = row[8:12]
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)
        
# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\WWR_v01_global_array_1981-1990.txt', r'data\intermediate\1981_1990_weather_data.csv')

In [58]:
# Check results
weather_data_1981_1990 = pd.read_csv(r'data\intermediate\1981_1990_weather_data.csv')
weather_data_1981_1990.head()

C:\Users\George Jieh\AppData\Local\Temp\ipykernel_12868\3247821989.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_data_1981_1990 = pd.read_csv(r'data\intermediate\1981_1990_weather_data.csv')


,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,70454,ALASKA,ADAK,4,1981,-0.9,0.7,1.9,4.2,6.6,7.8,10.7,11.4,8.8,6.7,2.9,1.3
1,70454,ALASKA,ADAK,4,1982,0.6,0.9,2.9,2.9,5.0,7.3,8.7,10.4,8.3,6.6,3.6,-0.4
2,70454,ALASKA,ADAK,4,1983,-1.5,-0.1,1.2,3.1,5.2,7.4,9.6,12.2,9.1,6.7,2.3,2.3
3,70454,ALASKA,ADAK,4,1984,0.6,-1.1,2.4,2.9,5.8,8.7,11.0,11.7,11.0,7.3,3.9,3.1
4,70454,ALASKA,ADAK,4,1985,1.9,1.4,-0.4,3.1,5.4,7.4,10.0,10.2,9.3,6.5,5.1,2.8


In [51]:
# 1991-2000 Weather Data Parsing
def parse_value(row, start, data_type):
    value_str = row[start:start+5].replace('/', '').strip()  # Remove slashes
    if '-' in value_str:
        # If there's a dash, remove it and treat the number as negative
        value_str = value_str.replace('-', '')
        value = -float(value_str) if value_str else None
    else:
        value = float(value_str) if value_str else None
    
    return round(value * 0.1, 1) if value is not None and data_type != "1" else value

def parse_row(row, station_info):
    station_id = row[2:7]
    data_type = row[7]
    if data_type == "1":
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = row[8:12]
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)
        
# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\WWR_1991-2000.txt', r'data\intermediate\1991_2000_weather_data.csv')

In [57]:
# Check results
weather_data_1991_2000 = pd.read_csv(r'data\intermediate\1991_2000_weather_data.csv')
weather_data_1991_2000.head()

C:\Users\George Jieh\AppData\Local\Temp\ipykernel_12868\1410486960.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_data_1991_2000 = pd.read_csv(r'data\intermediate\1991_2000_weather_data.csv')


,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,62795,SUDAN,ABU NAAMA,5,1991,0.0,0.0,0.0,5.7,142.1,67.6,156.3,98.2,4.3,51.3,0.0,0.0
1,62795,SUDAN,ABU NAAMA,5,1992,0.0,0.0,0.0,0.0,24.8,25.2,174.5,173.7,61.8,37.9,3.5,0.0
2,62795,SUDAN,ABU NAAMA,5,1993,0.0,0.0,0.0,5.5,36.9,186.2,345.2,235.5,103.2,31.6,10.0,0.0
3,62795,SUDAN,ABU NAAMA,5,1994,0.0,0.0,0.0,0.0,31.2,18.3,192.2,235.3,78.5,8.0,0.0,0.0
4,62795,SUDAN,ABU NAAMA,5,1995,0.0,0.0,19.4,0.0,66.2,93.2,200.4,86.7,50.8,6.0,0.0,0.0


In [60]:
# 2001-2010 Weather Data Parsing
def parse_value(row, start, data_type):
    value_str = row[start:start+5].replace('/', '').strip()  # Remove slashes
    if '0T' in value_str:
        return None  # Return None if '0T' is found
    elif '-' in value_str:
        # If there's a dash, remove it and treat the number as negative
        value_str = value_str.replace('-', '')
        value = -float(value_str) if value_str else None
    else:
        value = float(value_str) if value_str else None
    
    return round(value * 0.1, 1) if value is not None and data_type != "1" else value

def parse_row(row, station_info):
    station_id = row[2:7]
    data_type = row[7]
    if data_type == "1":
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = row[8:12]
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)
        
# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\WWR_Region00_2001-2010.txt', r'data\intermediate\2001_2010_weather_data.csv')

In [62]:
# Check results
weather_data_2001_2010 = pd.read_csv(r'data\intermediate\2001_2010_weather_data.csv', encoding='ISO-8859-1')
weather_data_2001_2010.head()

,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1001,NORWAY,JAN MAYEN,4,2001,-2.1,-6.1,-7.3,-4.2,0.0,2.2,4.9,6.5,4.5,2.0,-3.3,-1.5
1,1001,NORWAY,JAN MAYEN,4,2002,-3.7,-5.7,-4.6,-0.9,1.3,4.8,6.9,7.2,4.4,2.7,2.0,0.4
2,1001,NORWAY,JAN MAYEN,4,2003,-6.9,-0.9,-2.8,-1.9,-0.6,3.6,5.9,6.9,4.9,-0.4,1.2,-5.1
3,1001,NORWAY,JAN MAYEN,4,2004,-3.4,-5.2,-0.7,-0.3,0.9,3.2,6.3,7.4,5.8,1.8,-2.3,-3.6
4,1001,NORWAY,JAN MAYEN,4,2005,-1.9,-3.0,-2.5,-0.5,-0.5,3.3,6.1,6.9,2.6,-0.1,-2.0,-0.8


In [71]:
# 2011-2016 Weather Data Parsing
def parse_value(row, start, data_type):
    for offset in range(5):  # Try shifting up to 4 indices to find a valid number
        try:
            value_str = row[start + offset:start + 5].replace('/', '').strip()
            if '0T' in value_str:
                return None  # Return None if '0T' is found
            elif '-' in value_str:
                # If there's a dash, remove it and treat the number as negative
                value_str = value_str.replace('-', '')
                value = -float(value_str) if value_str else None
            else:
                value = float(value_str) if value_str else None
            
            return round(value * 0.1, 1) if value is not None and data_type != 1 else value
        except ValueError:
            continue  # If ValueError, shift reading of substring and retry

    return None  # Return None if no valid number is found

def parse_row(row, station_info):
    if "\t" in row:
        # Return a row full of null values if tab character is found
        return [None] * 17

    station_id = int(row[2:7])
    data_type = int(row[7])
    if data_type == 1:
        country = row[19:42].strip()
        city = row[43:69].strip()
        station_info[station_id] = [country, city]
        return None  # No further data processing for data type 1

    else:
        year = int(row[8:12])
        values = [parse_value(row, i, data_type) for i in range(13, 73, 5)]
        country_city = station_info.get(station_id, ["", ""])
        return [station_id] + country_city + [data_type, year] + values

def process_file(input_file, output_file):
    columns = ["Station ID", "Country", "City", "Data Type", "Year", 
               "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

    station_info = {}
    data_rows = []

    with open(input_file, 'r') as file:
        for row in file:
            parsed_row = parse_row(row, station_info)
            if parsed_row:
                data_rows.append(parsed_row)

    # Write headers and data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)
        writer.writerows(data_rows)

# Replace with desired file path. First path is the input and the second path is the output
process_file(r'data\raw\WWR_Region00_2011-2016.txt', r'data\intermediate\2011_2016_weather_data.csv')

In [74]:
# Check results
weather_data_2011_2016 = pd.read_csv(r'data\intermediate\2011_2016_weather_data.csv', encoding='ISO-8859-1')
weather_data_2011_2016.head()

,Station ID,Country,City,Data Type,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1001.0,NORWAY,JAN MAYEN,4.0,2011.0,-3.3,-0.6,-5.9,-0.5,1.0,3.5,5.3,6.1,5.9,2.1,1.3,-3.3
1,1001.0,NORWAY,JAN MAYEN,4.0,2012.0,0.5,-3.7,-2.3,-4.5,-0.6,2.4,4.8,5.9,3.3,1.0,-1.1,-2.1
2,1001.0,NORWAY,JAN MAYEN,5.0,2011.0,112.2,99.3,65.7,83.9,30.3,27.5,12.6,34.7,108.5,79.1,153.3,56.7
3,1001.0,NORWAY,JAN MAYEN,5.0,2012.0,61.7,78.0,59.5,12.0,24.8,6.5,18.9,32.2,77.0,34.9,92.1,42.8
4,2013.0,SWEDEN,RITSEM A,2.0,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
